In [1]:
import pandas as pd

data = pd.read_csv("clean_weather.csv", index_col=0)

In [2]:
data = data.ffill()

In [3]:
train = data[["tmax", "tmin", "rain"]].to_numpy()[:-1000,:]
target = data[["tmax_tomorrow"]].to_numpy()[:-1000,:]

test = data[["tmax", "tmin", "rain"]].to_numpy()[-1000:,:]
test_target = data[["tmax_tomorrow"]].to_numpy()[-1000:,:]

In [4]:
import numpy as np

In [5]:
def mse_loss(predictions, actuals):
    return ((actuals - predictions) ** 2)

In [6]:
def finite_diff(func, x):
    eps = 1e-7
    grad = []
    for i in range(len(x)):
        row = np.zeros((1,len(x)))
        row[0,i] = 1
        grad.append((func(x + eps * row) - func(x - eps * row)) / (2 * eps))
    grad = np.array(grad).reshape(len(x), 1)
    print(grad)
    return grad

In [7]:
class Layer():
    def __init__(self, in_shape, out_shape, relu=True):
        self.weight = np.random.rand(in_shape, out_shape)
        self.bias = np.ones((1, out_shape))
        self.relu = relu
        self.hidden = None
        self.activated = None
    
    def forward(self, x, relu=True):
        linear = np.matmul(x, self.weight) + self.bias
        self.hidden = linear
        if self.relu and relu:
            relu = np.maximum(linear, np.zeros(linear.shape))
            self.activated = relu
            return relu
        return linear
    
    def backward(self, gradient):
        pass

In [8]:
class Network():
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, x):
        output = np.zeros((x.shape[0],1))
        for i in range(x.shape[0]):
            vals = x[i:(i+1),:].copy()
            for layer in self.layers:
                vals = layer.forward(vals)
            output[i,:] = vals
        return output
    
    def backward(self):
        pass

In [9]:
layers = [
    Layer(3, 10, True),
    Layer(10, 1, False)
]

net = Network(layers)

In [10]:
epochs = 100
lr = 1e-7

for epoch in range(epochs):
    predictions = net.forward(train)
    
    print(f"Epoch: {epoch} MSE: {np.mean(mse_loss(predictions, target))}")
    outer_grad = predictions - target
    
    # Calc gradients

    for i in np.random.randint(0, outer_grad.shape[0], 64):
        elem = outer_grad[i,:].reshape((1,1))

        l2_start_grad = np.multiply(elem, np.heaviside(net.layers[1].hidden,1))
        l2_w_grad = np.matmul(l2_start_grad, net.layers[0].hidden).T
        l2_b_grad = l2_start_grad
        l2_grad = np.matmul(net.layers[1].weight, l2_start_grad).T
        
        l1_start_grad = np.multiply(l2_grad, np.heaviside(net.layers[0].hidden,1))
        l1_w_grad = np.matmul(l1_start_grad.T, train[1:2,:]).T
        l1_b_grad = l1_start_grad
        
        
        net.layers[1].weight -= l2_w_grad * lr
        net.layers[1].bias -= l2_b_grad * lr

        net.layers[0].weight -= l1_w_grad * lr
        net.layers[0].bias -= l1_b_grad * lr

Epoch: 0 MSE: 56619.31049568112
Epoch: 1 MSE: 20639.287422557045
Epoch: 2 MSE: 8032.496752936202
Epoch: 3 MSE: 3496.975684837547
Epoch: 4 MSE: 1575.4213051317417
Epoch: 5 MSE: 745.2409618466771
Epoch: 6 MSE: 367.1542284095236
Epoch: 7 MSE: 197.0012187514792
Epoch: 8 MSE: 96.53467831317077
Epoch: 9 MSE: 57.77916667256173
Epoch: 10 MSE: 40.13826563873698
Epoch: 11 MSE: 31.653651138440065
Epoch: 12 MSE: 26.159070303471143
Epoch: 13 MSE: 26.336156169205434
Epoch: 14 MSE: 23.801978651775755
Epoch: 15 MSE: 23.42056794205556
Epoch: 16 MSE: 23.324244666426033
Epoch: 17 MSE: 23.34619769897153
Epoch: 18 MSE: 23.38199706956884
Epoch: 19 MSE: 23.273900375187274
Epoch: 20 MSE: 23.3520621180795
Epoch: 21 MSE: 23.24289047240123
Epoch: 22 MSE: 23.25054381660914
Epoch: 23 MSE: 23.30042992911694
Epoch: 24 MSE: 23.396687317153006
Epoch: 25 MSE: 23.354232362239983
Epoch: 26 MSE: 23.242504547651254
Epoch: 27 MSE: 23.25503247455426
Epoch: 28 MSE: 23.23605750861873
Epoch: 29 MSE: 23.272323194067372
Epoch: 30

In [11]:
test_preds = net.forward(test)

In [12]:
np.mean(mse_loss(test_preds, test_target))

25.293393446085258